In [1]:
import pandas as pd
import networkx as nx
import gurobipy as gp
from gurobipy import GRB

In [13]:
protein1 = {1: [2],
            2: [1, 9],
            3: [4, 5],
            4: [3],
            5: [3, 6],
            6: [5, 7],
            7: [6, 9],
            8: [9],
            9: [2, 7, 8]}

protein2 = {1: [4],
            2: [3],
            3: [2],
            4: [1, 6, 7],
            5: [6], 
            6: [4, 5, 8],
            7: [4, 8, 10],
            8: [6, 7],
            9: [10],
           10: [7, 9, 11],
           11: [10]}

P1 = nx.Graph()
P1.add_nodes_from(protein1.keys())
P1.add_edges_from([(i, j) for i in protein1.keys() 
                          for j in protein1[i]])

P2 = nx.Graph()
P2.add_nodes_from(protein2.keys())
P2.add_edges_from([(i, j) for i in protein2.keys() 
                          for j in protein2[i]])

In [18]:
ij = [(i, j) for i in P1.nodes() for j in P2.nodes()]
ijkl = [(i, j, k, l) for i, k in P1.edges() for j, l in P2.edges()]
no_cross = [(i,j,k,l) for i,j in ij for k,l in ij if (i<k) and (l<j)]

In [53]:
model = gp.Model('Protein Comparison')

# add vars
x = model.addVars(ij,
                  vtype=GRB.BINARY,
                  name='x')

w = model.addVars(ijkl,
                  vtype=GRB.BINARY,
                  name='w')

# objective function
model.setObjective((gp.quicksum(w[e] for e in ijkl)),
                   GRB.MAXIMIZE)

# add constraints
model.addConstrs((gp.quicksum(x[(i,j)] for j in P2.nodes()) <= 1) for i in P1.nodes())

model.addConstrs((gp.quicksum(x[(i,j)] for i in P1.nodes()) <= 1) for j in P2.nodes())

for i, k in P1.edges():
    for j, l in P2.edges():
        model.addConstr((w[i,j,k,l] <= x[i,j]))
        model.addConstr((w[i,j,k,l] <= x[k,l]))

model.addConstrs((x[i,j] + x[k,l] <= 1 for i, j, k, l in no_cross))

model.update()
model.write('Protein Comparison.lp')

In [54]:
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2160 rows, 179 columns and 4478 nonzeros
Model fingerprint: 0xfa76410d
Variable types: 0 continuous, 179 integer (179 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1940 rows and 66 columns
Presolve time: 0.05s
Presolved: 220 rows, 113 columns, 1692 nonzeros
Found heuristic solution: objective 2.0000000
Variable types: 0 continuous, 113 integer (113 binary)

Root relaxation: objective 6.750000e+00, 167 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.75000    0   43    2.00000    6.75000   238%     -    0s
H   

In [61]:
print('Maximum number of edges matched: {}'.format(model.objVal))

for i,j,k,l in w:
    if w[i,j,k,l].x > 0:
        print('Edge ({},{}) of Protein 1 paired with ({},{}) of Protein 2'.format(i,k,j,l))

Maximum number of edges matched: 5.0
Edge (1,2) of Protein 1 paired with (2,3) of Protein 2
Edge (3,4) of Protein 1 paired with (4,6) of Protein 2
Edge (3,5) of Protein 1 paired with (4,7) of Protein 2
Edge (5,6) of Protein 1 paired with (7,10) of Protein 2
Edge (6,7) of Protein 1 paired with (10,11) of Protein 2
